In [99]:
# load env
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [100]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from typing import List
import os

def getRetriever(tool_url: str):
    print("Loading retriever for " + tool_url)
    key = tool_url.replace("/", "_").replace(":", "_")
    # embeddings = OllamaEmbeddings()
    embeddings = OpenAIEmbeddings()
    # if the vectorstore already exists, load it
    if os.path.exists("./vectorstores/" + key):
        vectorstore = FAISS.load_local(
            "./vectorstores/" + key, embeddings, index_name=key)
        retriever = vectorstore.as_retriever()
        return retriever
    # use the web base loader to get the search results
    loader = WebBaseLoader(tool_url)
    data = loader.load()
    # create a retriever out of the data
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    docs = text_splitter.split_documents(data)
    vectorstore = FAISS.from_documents(docs, embeddings)
    # store the embeddings locally
    vectorstore.save_local("./vectorstores/" + key, key)
    retriever = vectorstore.as_retriever()
    # get the search results
    return retriever

In [101]:
class Response(BaseModel):
    """Final response to the question being asked"""
    tool_name: str = Field(..., description="Name of the tool")
    web_url: str = Field(..., description="URL of the tool")
    tool_function: str = Field(..., description="Function of the tool")
    tool_features: str= Field(..., description="Features of the tool")
    business_function: str = Field(..., description="Business function of the tool")
    users: str = Field(..., description="Target users of the tool")
    pricing: str = Field(..., description="Pricing of the tool")
    hq_location_city: str = Field(..., description="Headquarters location of the tool")
    hq_location_country: str = Field(..., description="Headquarters location of the tool")
    hq_location_state: str = Field(..., description="Headquarters location of the tool")
    key_clients: str = Field(..., description="Key clients of the tool")

In [102]:
tool_urls=[
  "https://www.bardeen.ai/"
]

In [103]:
# custom parsing logic
import json
from langchain_core.agents import AgentActionMessageLog, AgentFinish


def parse(output):
    print("Output: ",output)
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [104]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage

representative = {'id': 1, 'tool_name': '10web.io', 'website': '10web.io', 'tool_function': 'Speech from text, Text from Speech', 'business_function': 'Sales, Recruiting, Marketing', 'industries_served': 'IT, E-commerce, Research, Media', 'years_in_operation': 'Less than one year', 'twitter': 'https://twitter.com/10Web_io', 'num_twitter': '12,300', 'linkenin': 'https://www.linkedin.com/company/10web/', 'num_linkedin': '7,001', 'num_employees': '51-250', 'revenue': '$10M-$50M', 'hq_location_city': 'San Francisco', 'hq_location_state': 'California', 'hq_location_country': 'US', 'key_clients': 'GoDaddy, HostGator, Wix, Amazon, Google', 'overview': 'The 10Web AI Website Builder is a tool that automates website building using the latest generative technologies. Build professional and original websites from the ground up in minutes, with content unique to your business using minimal resources.', 'users': 'Artists, Bloggers, Developers, Content Creators, Students, Gamers', 'pricing': 'AI Starter $5/month $60/annually\n AI Premium $8/month $90/annnually\n AI Ultimate $12/month $135/annually\n One week free trial', 'tool_features': 'AI website builder, AI ecommerce website builder, Automated hosting, Page speed booster, AI SEO tool, AI assistant, AI business name generator, AI marketing stategy generator', 'integrations': 'Whatsapp, Telegram, Facebook, Drive, Notion, Slack, Zapier', 'logo': 'https://logo.clearbit.com/10web.io'}

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a smart AI tools expert called Fortune Cookie. Use the tools available to you to answer as comprehensively as possible. If any field doesn't have sufficient information, you can fill it with 'Unspecified'."
        ),
        SystemMessage(
            content="An example tool response is provided for example: "+json.dumps(representative)
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [105]:
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai.chat_models import ChatOpenAI

# llm = ChatOllama(model="mistral", temperature=0)
# llm = OllamaFunctions(model="mistral")
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [106]:
# for each tool url, get the retriever
from langchain.tools.retriever import create_retriever_tool
for url in tool_urls:
    retriever = getRetriever(url)
    retriever_tool = create_retriever_tool(
        retriever,
        "tool-info-retriever",
        "Query this retriever to get any available information on the current tool",
    )
    llm_with_tools = llm.bind_functions([retriever_tool, Response])
    print("Bound llm with tools")
    agent = (
        {
            "input": lambda x: x["input"],
            # Format agent scratchpad from intermediate steps
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | parse
    )
    print("Created agent")
    agent_executor = AgentExecutor(
        tools=[retriever_tool], agent=agent, verbose=True)
    print("Created agent executor")
    agent_executor.invoke(
        {"input": "Tell me about Bardeen AI"}
    )

Loading retriever for https://www.bardeen.ai/
Bound llm with tools
Created agent
Created agent executor


> Entering new AgentExecutor chain...
Output:  content='' additional_kwargs={'function_call': {'arguments': '{\n  "query": "Bardeen AI"\n}', 'name': 'tool-info-retriever'}}
Bardeen AI: Workflow Task Automation Software

















Product

download the extensionFor ChromeJoin over 200k usersPlatform OverviewHow it worksInstall the extension, find a workflow, automate awayPlatform capabilitiesDiscover what AI-powered automations can do for youRelease notesGet the latest details on product releasesSETUP YOUR WORKFLOWMagic boxGenerate workflows with everyday languageWorkflow templatesThousands of workflows, ready to use in one clickNo-code builderAssemble workflows with customizable building blocksLatestN.E.A.T. selling in 2024: close more with AI workflow automationAccelerate NEAT selling in 2024. Bardeen can automate repetitive workflows. Our AI solves tedious tasks. Free forev